# 3D MAPS WITH PYDECK
For more information on pydeck, visit the python package index or visit the documentation at https://pydeck.gl.

# DOCUMENTATION

In [11]:
from IPython.display import IFrame
documentation = IFrame(src='https://pydeck.gl', width=1000, height=500)
display(documentation)

# NOTES

In [2]:
# Mapbox requires an API key
# command + scroll to rotate map (different on Windows and other combinations work also - shift + scroll)

# PATTERN TO CREATE MAPS
# data
# view (location, zoom level, etc.)
# layer
# render map
# display and save map (to_html(), show())

# IMPORTS

In [3]:
import pydeck as pdk
import pandas as pd
from vega_datasets import data as vds
import ipywidgets
# use palettable for colormap colors
from palettable.cartocolors.sequential import BrwnYl_3
import json
import PYTHON_API_KEYS

# enter your api key here
# API key required for mapbox basemap tiles
MAPBOX_API_KEY = PYTHON_API_KEYS.API_KEYS.MAPBOX

# To enable pydeck for Jupyter Notebook:
# jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
# jupyter nbextension enable --sys-prefix --py pydeck

# ARCLAYER EXAMPLE - WORK COMMUTE PATTERNS

examples of other map styles:
* mapbox://styles/mapbox/streets-v11
* mapbox://styles/mapbox/outdoors-v11
* mapbox://styles/mapbox/light-v10
* mapbox://styles/mapbox/dark-v10
* mapbox://styles/mapbox/satellite-v9
* mapbox://styles/mapbox/satellite-streets-v11

In [4]:
# ArcLayer - work commute patterns (create dataframe with fake data)
# to change look of commute lines use arguments such as pitch, get_width, get_tilt, etc.

# data
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/fake_commute_data.csv'
commute_pattern = pd.read_csv(data)
print(commute_pattern.head(2))

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=32.800382, longitude=-97.040728, pitch=50, zoom=9)

# layer
# from home (orange) to work (purple)
arc_layer = pdk.Layer('ArcLayer',
                      data=commute_pattern,
                      get_source_position=['from_lon', 'from_lat'],
                      get_target_position=['to_lon', 'to_lat'],
                      get_width=5,
                      get_tilt=15,
                      # RGBA colors (red, green, blue, alpha)
                      get_source_color=[255, 165, 0, 80],
                      get_target_color=[128, 0, 128, 80])

# render map
# choose map style
# mapbox api key required for basemap tile
arc_layer_map = pdk.Deck(map_style='mapbox://styles/mapbox/light-v10',
                         layers=arc_layer,
                         initial_view_state=view,
                         mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
arc_layer_map.to_html('/Users/ryannoonan/Desktop/arc_layer_map.html')
arc_layer_map.show()

    from_lat   from_lon     to_lat     to_lon
0  32.870715 -97.376346  32.769007 -96.812533
1  32.799592 -97.351823  32.822974 -96.801833


DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 32.800382, "longitude": -97.0…

# COLUMNLAYER EXAMPLE - POPULATION BY COUNTY

In [5]:
# data
# in this example, the population was scaled down
# to change visualization, try changing the elevation_scale and radius arguments

data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/us_county_centroids.csv'
county_population = pd.read_csv(data)
county_population['scaled_population'] = county_population['Population_2010']/1_000
print(county_population.columns)

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39.155726, longitude=-98.030561, pitch=50, zoom=3)

# layer
column_layer = pdk.Layer('ColumnLayer',
                         data=county_population,
                         get_position=['Longitude', 'Latitude'],
                         get_elevation='scaled_population',
                         elevation_scale=100,
                         radius=5000,
                         get_fill_color=[255, 165, 0, 80],
                         pickable=True,
                         auto_highlight=True)

# render map
# with no map_style, map goes to default
column_layer_map = pdk.Deck(layers=column_layer, 
                            initial_view_state=view,
                            mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
column_layer_map.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 39.155726, "longitude": -98.0…

# HEXAGONLAYER EXAMPLE - EARTHQUAKES

In [6]:
# data
# earthquakes data from kaggle
# the earthquakes data only includes latitude and longitude
earthquakes = pd.read_csv('earthquakes_data.csv')[['Longitude','Latitude']]
print(earthquakes.head())

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=31.315012, longitude=-30.712136, pitch=50, zoom=1)

# layer
hexagon_layer = pdk.Layer('HexagonLayer',
                          data=earthquakes,
                          get_position=['Longitude', 'Latitude'],
                          auto_highlight=True,
                          elevation_scale=50,
                          elevation_range=[0, 3_000],
                          radius=50_000,
                          extruded=True)

# render map
# with no map_style, map goes to default
hexagon_layer_map = pdk.Deck(layers=hexagon_layer, 
                             initial_view_state=view,
                             mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
hexagon_layer_map.show()

   Longitude  Latitude
0    145.616    19.246
1    127.352     1.863
2   -173.972   -20.579
3    -23.557   -59.076
4    126.427    11.938


DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 31.315012, "longitude": -30.7…

# HEATMAPLAYER EXAMPLE - POPULATION BY COUNTY

In [7]:
# data
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/us_population_by_city_2018.csv'
city_population = pd.read_csv(data)
print(city_population.head())

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39.155726, longitude=-98.030561, pitch=50, zoom=3)

# layer
heatmap_layer = pdk.Layer('HeatmapLayer',
                          data=city_population,
                          opacity=0.9,
                          get_position=['longitude', 'latitude'],
                          color_range=BrwnYl_3.colors,
                          threshold=0.2,
                          get_weight='Population_2018',
                          pickable=True)

# render map
heatmap_layer_map = pdk.Deck(layers=heatmap_layer,
                             initial_view_state=view,
                             mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
heatmap_layer_map.show()

          City       State  Population_2018  latitude  longitude
0     New York    New York          8398748   40.6635   -73.9387
1  Los Angeles  California          3990456   34.0194  -118.4108
2      Chicago    Illinois          2705994   41.8376   -87.6818
3      Houston       Texas          2325502   29.7866   -95.3909
4      Phoenix     Arizona          1660272   33.5722  -112.0901


DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 39.155726, "longitude": -98.0…

# GEOJSONLAYER EXAMPLE - POPULATION BY COUNTY

Shapefiles can be converted to geojson files using QGIS (EPSG 4326, WGS 84).

In [8]:
# data
geojson_data = 'https://raw.githubusercontent.com/groundhogday321/map-layers/main/indiana_counties.geojson'

# load file from computer
# f = open('indiana_geojson.geojson')
# geojson_data = json.load(f)

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39.782339, longitude=-86.160225, pitch=50, zoom=6)

# layer
geojson_layer = pdk.Layer('GeoJsonLayer',
                          data=geojson_data,
                          opacity=0.8,
                          stroked=False,
                          filled=True,
                          extruded=True,
                          wireframe=True,
                          pickable=True,
                          get_elevation='properties.POP2000 / 10',
                          get_fill_color=[255, 255, 255],
                          get_line_color=[255, 165, 0])

# render map
text_layer_map = pdk.Deck(map_style='mapbox://styles/mapbox/streets-v11',
                          layers=[geojson_layer],
                          initial_view_state=view,
                          mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
text_layer_map.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 39.782339, "longitude": -86.1…

# TEXTLAYER EXAMPLE - TEXTLAYER WITH GEOJSONLAYER

In [9]:
# data
geojson_data = 'https://raw.githubusercontent.com/groundhogday321/map-layers/main/texas_counties.geojson'

# load file from computer
# f = open('texas_counties.geojson')
# geojson_data = json.load(f)

data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/us_county_centroids.csv'
counties = pd.read_csv(data)
texas_counties = counties[counties['State'] == 'TEXAS'] 
print(texas_counties.columns)

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=31.917932, longitude=-98.664612, pitch=50, zoom=6)

# layer
geojson_layer = pdk.Layer('GeoJsonLayer',
                          data=geojson_data,
                          opacity=0.5,
                          stroked=False,
                          filled=True,
                          extruded=True,
                          wireframe=True,
                          get_fill_color=[255, 255, 255],
                          get_line_color=[255, 255, 255])

text_layer = pdk.Layer('TextLayer',
                       texas_counties,
                       pickable=True,
                       get_position=['Longitude', 'Latitude'],
                       get_text='County',
                       get_size=16,
                       get_angle=0,
                       get_color=[255, 255, 0])

# render map
text_layer_map = pdk.Deck(layers=[text_layer, geojson_layer],
                          initial_view_state=view,
                          mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
text_layer_map.show()

Index(['Sort', 'State_Abbreviation', 'State', 'FIPS', 'County',
       'Population_2010', 'Land_Area_KM_Squared', 'Land_Area_MI_Squared',
       'Water_Area_KM_Squared', 'Water_Area_MI_Squared',
       'Total_Area_KM_Squared', 'Total_Area_MI_Squared', 'Latitude',
       'Longitude'],
      dtype='object')


DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 31.917932, "longitude": -98.6…

# ANIMATED EXAMPLE
### ADD WIDGET FOR INTERACTIVE MAP WITH ANIMATION

In [10]:
# data
# in this example, the population was scaled down
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/gapminder_centroids.csv'
gapminder = pd.read_csv(data)
gapminder['scaled_population'] = gapminder['pop']/50_000

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39, longitude=35, pitch=50, zoom=1)

# layer
gapminder_layer = pdk.Layer('ColumnLayer',
                             data=gapminder,
                             get_position=['longitude', 'latitude'],
                             get_elevation='scaled_population',
                             elevation_scale=100,
                             radius=200_000,
                             get_fill_color=[255, 165, 0, 80],
                             pickable=True,
                             auto_highlight=True)

# render map
# with no map_style, map goes to default
gapminder_layer_map = pdk.Deck(layers=gapminder_layer, 
                               initial_view_state=view,
                               mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
# display(gapminder_layer_map.show())
# display at botton

# widgets
slider = ipywidgets.IntSlider(value=1955, min=1955, max=2005, step=5)
play = ipywidgets.Play(value=1955, min=1955, max=2005, step=5, description='Press play', interval=1_000)
ipywidgets.jslink((play, 'value'), (slider, 'value'))
layout = ipywidgets.HBox([slider, play])

# function
def update_plot(year):
    gapminder_layer.data = gapminder[gapminder['year'] == year]
    return gapminder_layer_map.update()

# interaction between widget and function
interact = ipywidgets.interactive_output(update_plot, {'year': slider});
display(layout, interact)

# display and save map (to_html(), show())
gapminder_layer_map.show()

Output()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 39, "longitude": 35, "pitch":…